In [2]:
pip install vit-keras


In [5]:
pip install tensorflow-addons


   ---------------------------------------- 0.0/719.8 kB ? eta -:--:--
   ----------- ---------------------------- 204.8/719.8 kB 6.3 MB/s eta 0:00:01
   ------------------------- -------------- 450.6/719.8 kB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 719.8/719.8 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install opencv-python-headless


   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
    --------------------------------------- 0.8/38.5 MB 7.7 MB/s eta 0:00:05
   - -------------------------------------- 1.8/38.5 MB 8.0 MB/s eta 0:00:05
   --- ------------------------------------ 3.0/38.5 MB 8.1 MB/s eta 0:00:05
   --- ------------------------------------ 3.7/38.5 MB 7.8 MB/s eta 0:00:05
   ---- ----------------------------------- 4.5/38.5 MB 7.6 MB/s eta 0:00:05
   ----- ---------------------------------- 5.3/38.5 MB 7.5 MB/s eta 0:00:05
   ------ --------------------------------- 6.5/38.5 MB 7.6 MB/s eta 0:00:05
   ------- -------------------------------- 7.4/38.5 MB 7.9 MB/s eta 0:00:04
   -------- ------------------------------- 8.6/38.5 MB 7.9 MB/s eta 0:00:04
   --------- ------------------------------ 9.1/38.5 MB 8.0 MB/s eta 0:00:04
   ---------- ----------------------------- 10.2/38.5 MB 8.0 MB/s eta 0:00:04
   ----------- ---------------------------- 11.0/38.5 MB 8.0 MB/s eta 0:00:04
   -

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0  # We will use this as a feature extractor

from vit_keras import vit


: 

In [ ]:
data_dir = 'M:\projects\Major_proj\Fruits_Dataset\Fruit-262'

# Image Data Generator for preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Use 20% of data for validation
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


: 

In [ ]:
# Load the ViT model
vit_model = vit.vit_b16(
    image_size=224,
    pretrained=True,
    include_top=False,
    pretrained_top=False
)

# Freeze the base model
vit_model.trainable = False

# Add custom layers on top
inputs = Input(shape=(224, 224, 3))
x = vit_model(inputs)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


: 

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5
)


: 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


: 

In [ ]:
model.save('fruit_classifier_vit.h5')


: 

: 

: 

: 

: 